In [ ]:
from __future__ import division
import glob
from math import exp

from collections import defaultdict, OrderedDict

def partial(fun, *args, **kwargs):
    """ functools.partial does not seem to preserve name attribute """
    from functools import partial as _partial
    f = _partial(fun, *args, **kwargs)
    f.name = fun.name
    return f

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
data = defaultdict(dict)
temps = []
for d in glob.glob('L*'):
    deciC = int(float(d[1:])*10)
    temps.append(deciC)
    for f in glob.glob(d+'/*.txt'):
        idx = int(f.split('/')[-1].split('.txt')[0])
        xy = np.genfromtxt(f)
        xy[:, 0] /= 1000.0  # ms -> s
        data[deciC][idx] = xy
temps = sorted(temps)

In [ ]:
def plot_abs(t, Abs, **kwargs):
    plt.plot(t, Abs, **kwargs)
    if kwargs.get('label', None) is not None:
        plt.legend(loc='best')
    plt.xlabel('t / s')
    plt.ylabel('Abs')
    
def plot_raw_data(deciC, idx, **kwargs):
    if 'label' not in kwargs:
        kwargs['label'] = '{} degC, #{}'.format(deciC/10, idx)
    t = data[deciC][idx][:,0]
    Abs = data[deciC][idx][:,1]
    plot_abs(t, Abs, **kwargs)

plot_raw_data(170, 1, alpha=0.3)
plot_raw_data(170, 2, alpha=0.3)
plot_raw_data(170, 3, alpha=0.3)
plot_raw_data(170, 4, alpha=0.3)
plot_raw_data(170, 5, alpha=0.3)
plot_raw_data(270, 1)
[data[190][idx].shape for idx in range(1,6)]

In [ ]:
# Experimental data:
eps = 5148 # 480nm, (per molar * per cm)
Z = 2e-3 / 2 # [SCN-] / molar
Y = 4e-3 / 2 # [Fe3+] / molar
K = 10**2.065 # beta1 / per molar

# Example of simple treatment (pseudo first order)
x, y = data[190][1][:, 0], data[190][1][:, 1]
yp = y[-1] - y
ymin = yp[0] * 0.05  # avoid taking logarithm of negative values
ymin_thresh_idx = np.argwhere(yp < ymin)[0]
plt.figure(figsize=(12, 4))
plt.subplot(1,2,1)
plt.plot(x[:ymin_thresh_idx], yp[:ymin_thresh_idx])
plt.plot(x[ymin_thresh_idx:], yp[ymin_thresh_idx:])

t = x[:ymin_thresh_idx]
lny = np.log(yp[:ymin_thresh_idx])
plt.subplot(1,2,2)
plt.plot(t, lny)
p = np.polyfit(t, lny, 1)
kf_fit = -p[0]/Y
plt.plot(t, np.polyval(p, t), label='$k_f$ = {0:.2f}'.format(kf_fit))
_ = plt.legend()

In [ ]:
def pseudo_irrev(t, kf, P0, t0, excess_C, limiting_C, eps_l):
    return P0*eps_l*limiting_C*(1 - np.exp(-excess_C*kf*(t-t0)))
pseudo_irrev.name = 'Pseudo first order irreversible'

def pseudo_rev(t, kf, P0, t0, excess_C, limiting_C, eps_l, beta):
    kb = kf/beta
    return P0*eps_l*limiting_C*excess_C*kf/(excess_C*kf + kb)*(1 - np.exp(-(excess_C*kf+kb)*(t-t0)))
pseudo_rev.name = 'Pseudo first order reversible'

def binary_irrev(t, kf, P0, t0, excess_C, limiting_C, eps_l):
    return P0*eps_l*excess_C*(1 - np.exp(-kf*(excess_C-limiting_C)*(t-t0)))/(excess_C/limiting_C - np.exp(-kf*(t-t0)*(excess_C-limiting_C)))
binary_irrev.name = 'Second order irreversible'

def binary_rev(t, kf, P0, t0, excess_C, limiting_C, eps_l, beta):
    kb = kf/beta
    a = kf
    b = -excess_C*kf - limiting_C*kf - kb
    c = excess_C*limiting_C*kf
    P = np.sqrt(b**2 - 4*a*c)
    Q = P + b
    R = P - b
    return P0*eps_l*Q*(1 - np.exp(P*(t-t0)))/(2*a*(Q/R + np.exp(P*(t-t0))))
binary_rev.name = 'Second order reversible'

In [ ]:
funcs = OrderedDict([
    ('pseudo_irrev', partial(pseudo_irrev, excess_C=Y, limiting_C=Z, eps_l=eps)),
    ('pseudo_rev', partial(pseudo_rev, excess_C=Y, limiting_C=Z, eps_l=eps, beta=K)),
    ('binary_irrev', partial(binary_irrev, excess_C=Y, limiting_C=Z, eps_l=eps)),
    ('binary_rev', partial(binary_rev, excess_C=Y, limiting_C=Z, eps_l=eps, beta=K))
])

In [ ]:
exp(-(-12e3 + 298.15*46.8)/8.314511/298.15)
exp(5.6e3/8.314511/298.15)

In [ ]:
t = np.linspace(0,1)
plt.plot(t, funcs['binary_irrev'](t, 800, 0.2, -5e-3))

In [ ]:
def fit_raw(fun, t, Abs, p0):
    popt, pcov = curve_fit(fun, t, Abs, p0)
    if pcov.shape != (len(p0), len(p0)):
        raise UserError("Optimization failed")
    residuals = Abs - fun(t, *popt)
    return popt, pcov, residuals

def fit_data(fun, tempC, idx, p0):
    t = data[tempC][idx][:, 0]
    Abs = data[tempC][idx][:, 1]
    return fit_raw(fun, t, Abs, p0)
    
def plot_fit_data(fun, tempC, idx, fitparams, **kwargs):
    popt, pcov, residuals = fitparams
    if 'label' not in kwargs:
        kwargs['label'] = '{0:1d}: $k_f$ = {1:.5g} $\pm$ {2:.5g}'.format(idx, popt[0], pcov[0,0])
    t = data[tempC][idx][:, 0]
    plt.plot(t, fun(t, *popt), **kwargs)
    plt.legend(loc='best')


In [ ]:
fun = funcs['pseudo_irrev']
deciC = 190
indices = range(1, 6)
fits = []
for idx in indices:
    fits.append(fit_data(fun, deciC, idx, [kf_fit, 0.1, 0.0]))
    plot_raw_data(deciC, idx, alpha=0.2, label=None)
    
for idx, fitparams in enumerate(fits, 1):
    plot_fit_data(fun, deciC, idx, fitparams)

_ = plt.title(fun.name + ' fit')

In [ ]:
def evaluate(tempC, guess):
    plt.figure(figsize=(10, 10))
    funcparams = []
    for fidx, func in enumerate(funcs.values(), 1):
        plt.subplot(2, 2, fidx)
        print(func.name)
        funcparams.append([])
        for idx in range(1, 6):
            funcparams[-1].append(fit_data(func, tempC, idx, guess))
            #print(funcparams[-1][-1][:2])
            plot_raw_data(tempC, idx, alpha=0.2, label=None)
            plot_fit_data(func, tempC, idx, funcparams[-1][-1])
        plt.title(func.name + ' ({0:2d} degC)'.format(tempC))
    return funcparams

In [ ]:
fitdata = {}
for tempC in temps:
    print(tempC)
    fitdata[tempC] = evaluate(tempC, [400.0, 0.2, -1e-3])
    plt.show()

In [ ]:
from collections import defaultdict
kf_val = defaultdict(list)
kf_err = defaultdict(list)
rmse = defaultdict(list)
for fidx in range(len(funcs)):
    for deciC in temps:
        params = fitdata[deciC][fidx] # second order
        tot = 0.0
        mse = 0.0
        for popt, pcov, res in params:
            tot += popt[0]
            mse += sum(res**2)/len(res)
        avg = tot / len(params)
        s2 = 0.0
        for popt, pcov, res in params:
            s2 += (avg - popt[0])**2
        kf_val[fidx].append(avg)
        kf_err[fidx].append(s2**0.5/(len(params)-1))
        rmse[fidx].append(mse**0.5)
    kf_val[fidx] = np.array(kf_val[fidx])
    kf_err[fidx] = np.array(kf_err[fidx])
    rmse[fidx] = np.array(rmse[fidx])

In [ ]:
kf_val[3], kf_err[3], [np.average(rmse[i]) for i in range(4)]

In [ ]:
[sum(kf_err[i]) for i in range(4)]

In [ ]:
from math import log10, exp
reci_T = 1.0/(273.15 + np.array(temps)/10)
plt.figure(figsize=(14, 8))
log_arr = {}
for fidx, func in enumerate(funcs.values()):
    plt.subplot(2, 2, fidx + 1)
    x, y = reci_T, np.log(kf_val[fidx])
    plt.plot(x, y, 'o')
    p, cov = np.polyfit(x, y, 1, cov=True)
    sA = (exp(p[1]+cov[1,1]**0.5)-exp(p[1]-cov[1,1]**0.5))/2
    Ea, A = -p[0]*8.314511, np.exp(p[1])
    pow10 = int(log10(A))
    log_arr[fidx] = (Ea, A)
    fmtstr = ('$E_a={0:5.1f} \pm {1:5.1f}$ $kJ/mol,$\n$'
    'A=({2:5.1f} \pm {3:5.1f}) \cdot 10^{{{4:d}}} M^{{-1}} s^{{-1}}$')
    lbl = fmtstr.format(Ea/1e3, cov[0,0]**0.5/1e3, A*10**-pow10, sA*10**-pow10, pow10)
    plt.plot(x, np.polyval(p, x), label=lbl)
    plt.title(func.name)
    plt.ylabel('ln($k_f$ / $M^{-1}s^{-1}$)')
    plt.xlabel('$T^{-1} / K^{-1}$')
    plt.legend()
plt.tight_layout()
    

In [ ]:
def Arrhenius(T, A, Ea):
    R = 8.314511
    return A*np.exp(-Ea/(R*T))

In [ ]:
T = 273.15+np.array(temps)/10
#popt, pcov = curve_fit(Arrhenius, T, kf_val, [Ea, A],  1.0/np.array(kf_err))
kf_val[0]

In [ ]:
Tspan = T[-1] - T[0]
Tplot = np.linspace(T[0] - 0.05*Tspan, T[-1] + 0.05*Tspan)
fmtstr = ('$E_a={0:5.1f} \pm {1:5.1f}$ $kJ/mol,$\n$'
    'A=({2:5.1f} \pm {3:5.1f}) \cdot 10^{{16}} M^{{-1}} s^{{-1}}$')
plt.figure(figsize=(14,8))
for fidx, func in enumerate(funcs):
    plt.subplot(2, 2, fidx + 1)
    plt.errorbar(T, kf_val[fidx], kf_err[fidx], marker='.', linestyle='None')
    Ea_guess, A_guess = log_arr[fidx]
    #plt.plot(T, Arrhenius(T, A_guess, Ea_guess))
    popt, pcov = curve_fit(Arrhenius, T, kf_val[fidx], [A_guess, Ea_guess],  1/kf_err[fidx]**2)
    lbl = fmtstr.format(popt[1]/1e3, pcov[1,1]**0.5/1e3, popt[0]/1e16, pcov[0,0]**0.5/1e16)
    plt.plot(Tplot, Arrhenius(Tplot, *popt), label=lbl)
    plt.xlabel('T / K')
    plt.ylabel('$k_f$ / $M^{-1} \cdot s^{-1}$')
    plt.legend(loc='best')
    plt.title(funcs[func].name)

plt.tight_layout()